# Django 部署(Nginx)



## 1.  安装Nginx


## 2. uWSGI安装

## 3. 基于uWSGI和nginx部署Django

- 接下来配置Django与uwsgi连接。此处，假定的我的django项目位置为：/home/htfeng/mysite


```
$ uwsgi --http :8001 --chdir /home/fnngj/pydj/myweb/ --wsgi-file myweb/wsgi.py --master --processes 4 --threads 2 --stats 127.0.0.1:9191
```

常用选项：

- http ： 协议类型和端口号

- processes ： 开启的进程数量

- workers ： 开启的进程数量，等同于processes（官网的说法是spawn the specified number ofworkers / processes）

- chdir ： 指定运行目录（chdir to specified directory before apps loading）

- wsgi-file ： 载入wsgi-file（load .wsgi file）

- stats ： 在指定的地址上，开启状态服务（enable the stats server on the specified address）

- threads ： 运行线程。由于GIL的存在，我觉得这个真心没啥用。（run each worker in prethreaded mode with the specified number of threads）

- master ： 允许主进程存在（enable master process）

- daemonize ： 使进程在后台运行，并将日志打到指定的日志文件或者udp服务器（daemonize uWSGI）。实际上最常用的，还是把运行记录输出到一个本地文件上。

- pidfile ： 指定pid文件的位置，记录主进程的pid号。

- vacuum ： 当服务器退出的时候自动清理环境，删除unix socket文件和pid文件（try to remove all of the generated file/sockets）

## 4.Nginx+uwsgi+Django  

接下来，我们要将三者结合起来。首先罗列一下项目的所需要的文件：

mysite/

├── manage.py

├── mysite/

│   ├── __init__.py

│   ├── settings.py

│   ├── urls.py

│   └── wsgi.py

└── mysite_uwsgi.ini

在我们通过Django创建myweb项目时，在子目录myweb下已经帮我们生成的 wsgi.py文件。所以，我们只需要再创建myweb_uwsgi.ini配置文件即可，当然，uwsgi支持多种类型的配置文件，如xml，ini等。此处，使用ini类型的配置。

```
# myweb_uwsgi.ini file
[uwsgi]

# Django-related settings

socket = :8000

# the base directory (full path)
chdir           = /home/htfeng/mysite

# Django s wsgi file
module          = mysite.wsgi

# process-related settings
# master
master          = true

# maximum number of worker processes
processes       = 4

# ... with appropriate permissions - may be needed
# chmod-socket    = 664
# clear environment on exit
vacuum          = true
```

- 这个配置，其实就相当于在上一小节中通过wsgi命令，后面跟一堆参数的方式，给文件化了。

　　socket  指定项目执行的端口号。

　　chdir   指定项目的目录。

　　module  myweb.wsgi ，可以这么来理解，对于mysite_uwsgi.ini文件来说，与它的平级的有一个mysite目录，这个目录下有一个wsgi.py文件。

　　其它几个参数，可以参考上一小节中参数的介绍。

- 接下来，切换到myweb项目目录下，通过uwsgi命令读取myweb_uwsgi.ini文件启动项目。

```
$ uwsgi --ini mysite_uwsgi.ini 
[uWSGI] getting INI configuration from mysite_uwsgi.ini
*** Starting uWSGI 2.0.13.1 (64bit) on [Mon Aug 15 15:14:03 2016] ***
compiled with version: 5.4.0 20160609 on 11 August 2016 15:28:27
os: Linux-4.4.0-31-generic #50-Ubuntu SMP Wed Jul 13 00:07:12 UTC 2016
nodename: htfeng
machine: x86_64
clock source: unix
pcre jit disabled
detected number of CPU cores: 8
current working directory: /home/htfeng/mysite
detected binary path: /home/htfeng/anaconda3/bin/uwsgi
chdir() to /home/htfeng/mysite
your processes number limit is 128250
your memory page size is 4096 bytes
detected max file descriptor number: 1024
lock engine: pthread robust mutexes
thunder lock: disabled (you can enable it with --thunder-lock)
uwsgi socket 0 bound to TCP address :8000 fd 3
Python version: 3.5.1 |Anaconda 4.1.0 (64-bit)| (default, Jun 15 2016, 15:32:45)  [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
*** Python threads support is disabled. You can enable it with --enable-threads ***
Python main interpreter initialized at 0x102a820
your server socket listen backlog is limited to 100 connections
your mercy for graceful operations on workers is 60 seconds
mapped 363840 bytes (355 KB) for 4 cores
*** Operational MODE: preforking ***
WSGI app 0 (mountpoint='') ready in 1 seconds on interpreter 0x102a820 pid: 3608 (default app)
*** uWSGI is running in multiple interpreter mode ***
spawned uWSGI master process (pid: 3608)
spawned uWSGI worker 1 (pid: 3610, cores: 1)
spawned uWSGI worker 2 (pid: 3611, cores: 1)
spawned uWSGI worker 3 (pid: 3612, cores: 1)
spawned uWSGI worker 4 (pid: 3613, cores: 1)
```

注意查看uwsgi的启动信息，如果有错，就要检查配置文件的参数是否设置有误。

- 再接下来要做的就是修改/etc/nginx/sites-enabled/default配置文件。打开/etc/nginx/sites-enabled/default文件，添加如下内容。 

```
server {
    listen         8099; 
    server_name    127.0.0.1 
    charset UTF-8;
    access_log      /var/log/nginx/mysite_access.log;
    error_log       /var/log/nginx/mysite_error.log;

    client_max_body_size 75M;

    location / { 
        include uwsgi_params;
        uwsgi_pass 127.0.0.1:8000;
        uwsgi_read_timeout 2;
    }   
    location /static {
        expires 30d;
        autoindex on; 
        add_header Cache-Control private;
        alias /home/htfeng/mysite/static/;
     }
 }
```

listen 指定的是nginx代理uwsgi对外的端口号。

server_name  网上大多资料都是设置的一个网址（例，www.example.com）， 我这里如果设置成网址无法访问，所以，指定的到了本机默认ip。

在进行配置的时候，我有个问题一直想不通。nginx到底是如何uwsgi产生关联。现在看来大概最主要的就是这两行配置。
```
include uwsgi_params;

uwsgi_pass 127.0.0.1:8000;
```
include 必须指定为uwsgi_params；而uwsgi_pass指的本机IP的端口与myweb_uwsgi.ini配置文件中的必须一直。

现在重新启动nginx，翻看上面重启动nginx的命令。然后，访问：http://127.0.0.1:8099/

通过这个IP和端口号的指向，请求应该是先到nginx的。如果你在页面上执行一些请求，就会看到，这些请求最终会转到uwsgi来处理。